In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 22.3 MB/s eta 0:00:00


In [ ]:
# Imports PIL module
import urllib.request
import os
from PIL import Image

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

In [ ]:
!pip install symspellpy
!pip install word2number
!pip install donut
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.4-cp310-cp310-linux_x86_64.whl size=187491 sha256=9ef25785deb846c1f037232686bcaa3d1941dab5d2769dbfeb411f75c8f94fbe
  Stored in directory: /root/.cache/pip/wheels/4c/0f/10/c20d67cd765ee5b3666d759a307241bba0663135d6ee1c0072
Successfully built editdistpy
  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5566 sha256=3f2cecb8e5e0f4298524d9a8b4c56662137145bd5464bfe7399e74e3e7c05c3a
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number
  Preparing metadata (setup.py) ... done
  Created wheel for 

In [ ]:
import pkg_resources
import num2words
from symspellpy import SymSpell
from word2number import w2n

from dateutil import relativedelta
from datetime import datetime
import re

In [ ]:
CHEQUE_PARSER_MODEL = "shivi/donut-cheque-parser"
TASK_PROMPT = "<parse-cheque>"


In [ ]:
import torch
import re
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"



In [ ]:
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image


In [ ]:
def load_donut_model_and_processor(trained_model_repo):
    donut_processor = DonutProcessor.from_pretrained(trained_model_repo)
    model = VisionEncoderDecoderModel.from_pretrained(trained_model_repo)
    model.to(device)
    return donut_processor, model

In [ ]:
def prepare_data_using_processor(donut_processor,image,task_prompt):
    ## Pass image through donut processor's feature extractor and retrieve image tensor
    pixel_values = donut_processor(image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    ## Pass task prompt for document (cheque) parsing task to donut processor's tokenizer and retrieve the input_ids
    decoder_input_ids = donut_processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]
    decoder_input_ids = decoder_input_ids.to(device)

    return pixel_values, decoder_input_ids

In [ ]:
def spell_check(amt_in_words):
    sym_spell = SymSpell(max_dictionary_edit_distance=2,prefix_length=7)
    dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_82_765.txt")
    bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

    suggestions = sym_spell.lookup_compound(amt_in_words, max_edit_distance=2)

    return suggestions[0].term

In [ ]:
def match_legal_and_courstesy_amount(legal_amount,courtesy_amount):
    macthing_amts = False
    if len(legal_amount) == 0:
        return macthing_amts

    corrected_amt_in_words = spell_check(legal_amount)
    print("corrected_amt_in_words:",corrected_amt_in_words)

    numeric_legal_amt = w2n.word_to_num(corrected_amt_in_words)

    print("numeric_legal_amt:",numeric_legal_amt)
    if int(numeric_legal_amt) == int(courtesy_amount):
        macthing_amts = True
    return macthing_amts


In [ ]:
def check_if_cheque_is_stale(cheque_issue_date):
    stale_check = False
    current_date = datetime.now().strftime('%d/%m/%y')
    current_date_ = datetime.strptime(current_date, "%d/%m/%y")
    cheque_issue_date_ = datetime.strptime(cheque_issue_date, "%d/%m/%y")
    relative_diff = relativedelta.relativedelta(current_date_, cheque_issue_date_)
    months_difference = (relative_diff.years * 12) + relative_diff.months
    print("months_difference:",months_difference)
    if months_difference > 3:
        stale_check = True
    return stale_check

In [ ]:
def parse_cheque_with_donut(input_image_path):

    image = load_image(input_image_path)
    donut_processor, model = load_donut_model_and_processor(CHEQUE_PARSER_MODEL)

    cheque_image_tensor, input_for_decoder = prepare_data_using_processor(donut_processor,image,TASK_PROMPT)

    outputs = model.generate(cheque_image_tensor,
                              decoder_input_ids=input_for_decoder,
                              max_length=model.decoder.config.max_position_embeddings,
                              early_stopping=True,
                              pad_token_id=donut_processor.tokenizer.pad_token_id,
                              eos_token_id=donut_processor.tokenizer.eos_token_id,
                              use_cache=True,
                              num_beams=1,
                              bad_words_ids=[[donut_processor.tokenizer.unk_token_id]],
                              return_dict_in_generate=True,
                              output_scores=True,)

    decoded_output_sequence = donut_processor.batch_decode(outputs.sequences)[0]

    extracted_cheque_details = decoded_output_sequence.replace(donut_processor.tokenizer.eos_token, "").replace(donut_processor.tokenizer.pad_token, "")
    ## remove task prompt from token sequence
    cleaned_cheque_details = re.sub(r"<.*?>", "", extracted_cheque_details, count=1).strip()
    ## generate ordered json sequence from output token sequence
    cheque_details_json = donut_processor.token2json(cleaned_cheque_details)
    print("cheque_details_json:",cheque_details_json['cheque_details'])

    ## extract required fields from predicted json

    amt_in_words  = cheque_details_json['cheque_details'][0]['amt_in_words']

    amt_in_figures = cheque_details_json['cheque_details'][1]['amt_in_figures']

    macthing_amts = match_legal_and_courstesy_amount(amt_in_words,amt_in_figures)

    payee_name = cheque_details_json['cheque_details'][2]['payee_name']

    bank_name = cheque_details_json['cheque_details'][3]['bank_name']

    cheque_date = cheque_details_json['cheque_details'][4]['cheque_date']

    stale_cheque = check_if_cheque_is_stale(cheque_date)

    return payee_name,amt_in_words,amt_in_figures,bank_name,cheque_date,macthing_amts,stale_cheque

In [ ]:
#install kaggle
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"darshannshenoy","key":"7f4f40f308b5933bce2244f7c3d4059d"}'}

In [ ]:
#create kaggle folder
! mkdir ~/.kaggle

In [ ]:
#copy kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

In [ ]:
#permission for json to act
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d medali1992/cheque-images

 95% 162M/171M [00:05<00:00, 19.3MB/s]
100% 171M/171M [00:05<00:00, 35.2MB/s]


In [ ]:
!unzip cheque-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: yolo x image/yolo x image/5499.jpg  
  inflating: yolo x image/yolo x image/55.jpg  
  inflating: yolo x image/yolo x image/550.jpg  
  inflating: yolo x image/yolo x image/5500.jpg  
  inflating: yolo x image/yolo x image/5501.jpg  
  inflating: yolo x image/yolo x image/5502.jpg  
  inflating: yolo x image/yolo x image/5503.jpg  
  inflating: yolo x image/yolo x image/5504.jpg  
  inflating: yolo x image/yolo x image/5505.jpg  
  inflating: yolo x image/yolo x image/5506.jpg  
  inflating: yolo x image/yolo x image/5507.jpg  
  inflating: yolo x image/yolo x image/5508.jpg  
  inflating: yolo x image/yolo x image/5509.jpg  
  inflating: yolo x image/yolo x image/551.jpg  
  inflating: yolo x image/yolo x image/5510.jpg  
  inflating: yolo x image/yolo x image/5511.jpg  
  inflating: yolo x image/yolo x image/5512.jpg  
  inflating: yolo x image/yolo x image/5513.jpg  
  inflating: yolo x image/yolo x image/5514.jpg  
  i

In [ ]:
result = parse_cheque_with_donut('/content/yolo x image/yolo x image/1000.jpg')

preprocessor_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/809M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:543: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
2024-04-26 01:02:53,275: E symspellpy.symspellpy] Dictionary file not found at /usr/local/lib/python3.10/dist-packages/symspellpy/frequency_dictionary_82_765.txt.
ERROR:symspellpy.symspellpy:Dictionary file not found at /usr/local/lib/python3.10/dist-packages/symspellpy/frequency_dictionary_82_765.txt.


cheque_details_json: [{'amt_in_words': 'Five Hundred and Thirty Eight'}, {'amt_in_figures': '538'}, {'payee_name': 'Seymour Schwab'}, {'bank_name': 'AXIS BANK'}, {'cheque_date': '06/05/22'}]
corrected_amt_in_words: five hundred and thirty eight
numeric_legal_amt: 538
months_difference: 23


In [ ]:
print(result)

('Seymour Schwab', 'Five Hundred and Thirty Eight', '538', 'AXIS BANK', '06/05/22', True, True)
